# ADS 509 Module 3: Group Comparison 

The task of comparing two groups of text is fundamental to textual analysis. There are innumerable applications: survey respondents from different segments of customers, speeches by different political parties, words used in Tweets by different constituencies, etc. In this assignment you will build code to effect comparisons between groups of text data, using the ideas learned in reading and lecture.

This assignment asks you to analyze the lyrics and Twitter descriptions for the two artists.

* Read in the data, normalize the text, and tokenize it. When you tokenize your Twitter descriptions, keep hashtags and emojis in your token set. 
* Calculate descriptive statistics on the two sets of lyrics and compare the results. 
* For each of the four corpora, find the words that are unique to that corpus. 
* Build word clouds for all four corpora. 

Each one of the analyses has a section dedicated to it below. Before beginning the analysis there is a section for you to read in the data and do your cleaning (tokenization and normalization). 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [1]:
import os
import re
import emoji
import pandas as pd

#!pip install wordcloud

from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
from wordcloud import WordCloud 

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer


In [2]:
# Use this space for any additional import statements you need

from lexical_diversity import lex_div as ld



In [3]:
# Place any addtional functions or constants you need here. 

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

# It's handy to have a full set of emojis
all_language_emojis = set()

for country in emoji.EMOJI_DATA : 
    for em in emoji.EMOJI_DATA[country] : 
        all_language_emojis.add(em)

# and now our functions
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """

    
    
    num_tokens = len(tokens)
    
    num_unique_tokens = len(set(tokens))
    
    lexical_diversity = ld.ttr(tokens)
    
    num_characters = 0
    for word in tokens:
        num_characters+= len(word)
        
    common_tokens =  Counter(tokens)
    top_five_token = common_tokens .most_common(5)
    
    
    if verbose == True:        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        
        print(f"The five most common tokens are : {top_five_token}")
       
          
        
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters,top_five_token])
    
    
   

def contains_emoji(s):
    
    s = str(s)
    emojis = [ch for ch in s if emoji.is_emoji(ch)]

    return(len(emojis) > 0)


def remove_stop(tokens) :
    
    # modify this function to remove stopwords
    return [w for w in tokens if w.lower() not in sw]
    
    

 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))



def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    return(text.split())


def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens) 




## Data Ingestion

Use this section to ingest your data into the data structures you plan to use. Typically this will be a dictionary or a pandas DataFrame.

In [4]:
# Feel free to use the below cells as an example or read in the data in a way you prefer


data_location = "/Users/fatemehkhosravi/Desktop/Assignment3/"
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

artist_files = {'cher':'cher_followers_data.txt',
                'robyn':'robynkonichiwa_followers_data.txt'}


In [5]:
twitter_data = pd.read_csv(data_location + twitter_folder + artist_files['cher'],
                           sep="\t",
                           quoting=3)

twitter_data['artist'] = "cher"

#twitter_data.head()

In [6]:
twitter_data_2 = pd.read_csv(data_location + twitter_folder + artist_files['robyn'],
                             sep="\t",
                             quoting=3)
twitter_data_2['artist'] = "robyn"

twitter_data = pd.concat([
    twitter_data,twitter_data_2])
    
del(twitter_data_2)


In [7]:
# read in the lyrics here

lyrics_data= {"artist":[], "song":[],"lyrics": []}
path1=data_location+lyrics_folder

for artist in os.listdir(path1):
    for song in os.listdir(os.path.join (path1+artist)):
        song_name= song.split("_")[1].split(".")[0]
        with open (os.path.join(path1+artist, song),'r') as f:
            lyric=f.read()
        
        lyrics_data['artist'].append(artist)
        lyrics_data['song'].append(song_name)
        lyrics_data['lyrics'].append(lyric)

lyrics_data=pd.DataFrame(lyrics_data) 
#lyrics_data

## Tokenization and Normalization

In this next section, tokenize and normalize your data. We recommend the following cleaning. 

**Lyrics** 

* Remove song titles
* Casefold to lowercase
* Remove stopwords (optional)
* Remove punctuation
* Split on whitespace

Removal of stopwords is up to you. Your descriptive statistic comparison will be different if you include stopwords, though TF-IDF should still find interesting features for you. Note that we remove stopwords before removing punctuation because the stopword set includes punctuation.

**Twitter Descriptions** 

* Casefold to lowercase
* Remove stopwords
* Remove punctuation other than emojis or hashtags
* Split on whitespace

Removing stopwords seems sensible for the Twitter description data. Remember to leave in emojis and hashtags, since you analyze those. 

In [8]:
# apply the `pipeline` techniques from BTAP Ch 1 or 5

my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

lyrics_data["tokens"] = lyrics_data["lyrics"].apply(prepare,pipeline=my_pipeline)
lyrics_data["num_tokens"] = lyrics_data["tokens"].map(len) 

twitter_data["tokens"] = twitter_data["description"].apply(prepare,pipeline=my_pipeline)
twitter_data["num_tokens"] = twitter_data["tokens"].map(len) 


In [9]:
twitter_data['has_emoji'] = twitter_data["description"].apply(contains_emoji)

Let's take a quick look at some descriptions with emojis.

In [10]:
twitter_data[twitter_data.has_emoji].sample(10)[["artist","description","tokens"]]

,artist,description,tokens
1466440,cher,certified trash™ feminist,"[certified, trash™, feminist]"
2097405,cher,αℓмσѕт ιѕ иєνєя єиσυggggн ! ✌️✌️,"[αℓмσѕт, ιѕ, иєνєя, єиσυggggн, ✌️✌️]"
711100,cher,🖤🖤✋🏻✋🏻𝕸𝖊𝖒𝖊𝖓𝖙𝖔 𝕬𝖀𝕯𝕰𝕽𝕰 𝕾𝖊𝖒𝖕𝖊𝖗! 🇮🇹🇮🇹MEGLIO MORIRE...,"[🖤🖤✋🏻✋🏻𝕸𝖊𝖒𝖊𝖓𝖙𝖔, 𝕬𝖀𝕯𝕰𝕽𝕰, 𝕾𝖊𝖒𝖕𝖊𝖗, 🇮🇹🇮🇹meglio, mo..."
153891,robyn,Stars can't shine without darkness ⭐,"[stars, cant, shine, without, darkness, ⭐]"
1728747,cher,🌊🗽save our Republic 2020,"[🌊🗽save, republic, 2020]"
454765,cher,"Dad to Austin & Hannah, Lover to an Awesome GF...","[dad, austin, hannah, lover, awesome, gf, laki..."
265725,cher,Olympic level under achiever / over thinker 😄....,"[olympic, level, achiever, thinker, 😄, mum, pa..."
1833774,cher,@jeffshakeit fan account ✨loyle to my capo✨,"[jeffshakeit, fan, account, ✨loyle, capo✨]"
1345478,cher,"Love family, country, NY, USA ❣️#Resist #VoteB...","[love, family, country, ny, usa, ❣️#resist, #v..."
304382,cher,Just a Atlanta Raised lady maneuvering through...,"[atlanta, raised, lady, maneuvering, thing, ca..."


With the data processed, we can now start work on the assignment questions. 

Q: What is one area of improvement to your tokenization that you could theoretically carry out? (No need to actually do it; let's not make perfect the enemy of good enough.)

A: Detecting other languages in twitter data and exclude the stop words for those languages. applying white space between tokens and emojis can be another improvement.  

## Calculate descriptive statistics on the two sets of lyrics and compare the results. 


In [11]:
#cher lyrics

song_cher=lyrics_data.loc[lyrics_data['artist']=='cher']
song_cher=list(song_cher['tokens'])

# convert nested list to a flat list
flat_list_cher = [element for innerList in song_cher for element in innerList]

assert(descriptive_stats(flat_list_cher, verbose=True))


There are 35916 tokens in the data.
There are 3703 unique tokens in the data.
There are 172634 characters in the data.
The lexical diversity is 0.103 in the data.
The five most common tokens are : [('love', 1004), ('im', 513), ('know', 486), ('dont', 440), ('youre', 333)]


In [12]:
# robyn lyrics

song_robyn=lyrics_data.loc[lyrics_data['artist']=='robyn']
song_robyn=list(song_robyn['tokens'])

# convert nested list to a flat list
flat_list_robyn = [element for innerList in song_robyn for element in innerList]

assert(descriptive_stats(flat_list_robyn, verbose=True))

There are 15227 tokens in the data.
There are 2156 unique tokens in the data.
There are 73787 characters in the data.
The lexical diversity is 0.142 in the data.
The five most common tokens are : [('know', 308), ('dont', 301), ('im', 299), ('love', 275), ('got', 251)]


Q: what observations do you make about these data? 

A: Among 5 top words in both lyrics,  3 of them are similar: “love”, “don’t”, “know” .  Lexical diversity for Cher’s lyrics is about 0.103 and for Robyn is about 0.142. Higher value means that the lyrics contain many different word types while lower value of lexical diversity refers to the repeated words or phrases. I am not familiar with Robyn’s songs, but I know Cher’s songs are usually similar to each other, simple and easy to memorize. 0.103 also proves my assumption about Cher's songs.


## Find tokens uniquely related to a corpus

Typically we would use TF-IDF to find unique tokens in documents. Unfortunately, we either have too few documents (if we view each data source as a single document) or too many (if we view each description as a separate document). In the latter case, our problem will be that descriptions tend to be short, so our matrix would be too sparse to support analysis. 

To avoid these problems, we will create a custom statistic to identify words that are uniquely related to each corpus. The idea is to find words that occur often in one corpus and infrequently in the other(s). Since corpora can be of different lengths, we will focus on the _concentration_ of tokens within a corpus. "Concentration" is simply the count of the token divided by the total corpus length. For instance, if a corpus had length 100,000 and a word appeared 1,000 times, then the concentration would be $\frac{1000}{100000} = 0.01$. If the same token had a concentration of $0.005$ in another corpus, then the concentration ratio would be $\frac{0.01}{0.005} = 2$. Very rare words can easily create infinite ratios, so you will also add a cutoff to your code so that a token must appear at least $n$ times for you to return it. 

An example of these calculations can be found in [this spreadsheet](https://docs.google.com/spreadsheets/d/1P87fkyslJhqXFnfYezNYrDrXp_GS8gwSATsZymv-9ms). Please don't hesitate to ask questions if this is confusing. 

In this section find 10 tokens for each of your four corpora that meet the following criteria: 

1. The token appears at least `n` times in all corpora
1. The tokens are in the top 10 for the highest ratio of appearances in a given corpora vs appearances in other corpora.

You will choose a cutoff for yourself based on the side of the corpus you're working with. If you're working with the Robyn-Cher corpora provided, `n=5` seems to perform reasonably well.

In [26]:
# cher lyrics

cher_count_corpora={'token':[], 'corpus1 count':[], 'concentration1':[]}
cher_tokens=lyrics_data.loc[lyrics_data['artist']=='cher','tokens']
cher_tokens=list(cher_tokens)
cher_tokens=[element for innerList in cher_tokens for element in innerList]
cher_counter=Counter(cher_tokens)

for t in cher_counter:
        if cher_counter[t] >=5:
            cher_count_corpora['token'].append(t)
            cher_count_corpora['corpus1 count'].append(cher_counter[t])
            concentration1 = cher_counter[t]/len(cher_tokens)
            cher_count_corpora['concentration1'].append(concentration1)


cher_count_corpora_df=pd.DataFrame(cher_count_corpora)

  
#robyn lyrics 

robyn_count_corpora={'token':[], 'corpus2 count':[], 'concentration2':[]}
robyn_tokens=lyrics_data.loc[lyrics_data['artist']=='robyn','tokens']
robyn_tokens=list(robyn_tokens)
robyn_tokens=[element for innerList in robyn_tokens for element in innerList]
robyn_counter=Counter(robyn_tokens)

for t in robyn_counter:
        if robyn_counter[t] >=5:
            robyn_count_corpora['token'].append(t)
            robyn_count_corpora['corpus2 count'].append(robyn_counter[t])
            concentration2 = robyn_counter[t]/len(robyn_tokens)
            robyn_count_corpora['concentration2'].append(concentration2)


robyn_count_corpora_df=pd.DataFrame(robyn_count_corpora)


#cher_count_corpora_df.shape
#robyn_count_corpora_df.shape



# merge two data frames

final_count_corpora_df= cher_count_corpora_df.merge(robyn_count_corpora_df, on ='token')


#ratio

final_count_corpora_df['concentration_ratio']=final_count_corpora_df['concentration1']/final_count_corpora_df['concentration2']

    

In [27]:
final_count_corpora_df.head()

,token,corpus1 count,concentration1,corpus2 count,concentration2,concentration_ratio
0,come,270,0.007518,72,0.004728,1.589855
1,stay,78,0.002172,15,0.000985,2.204600
2,ill,170,0.004733,73,0.004794,0.987308
3,away,196,0.005457,39,0.002561,2.130678
4,life,124,0.003453,39,0.002561,1.347980


In [28]:
#sort data frame based on 'concentration ratio ' column in descending order.
#The tokens are in the top 10 for the highest ratio of appearances in cher lyrics corpora 
#vs appearances in robyn lyrics corpora.

final_count_corpora_df.sort_values('concentration_ratio', ascending =False)[0:10]

,token,corpus1 count,concentration1,corpus2 count,concentration2,concentration_ratio
117,walk,118,0.003285,5,0.000328,10.005491
188,find,137,0.003814,6,0.000394,9.680453
183,enough,129,0.003592,7,0.000460,7.813004
66,man,213,0.005931,12,0.000788,7.525316
114,without,88,0.002450,5,0.000328,7.461722
218,strong,77,0.002144,5,0.000328,6.529007
21,live,68,0.001893,5,0.000328,5.765876
119,believe,145,0.004037,11,0.000722,5.588583
269,tears,57,0.001587,5,0.000328,4.833161
110,name,52,0.001448,5,0.000328,4.409199


In [29]:
#sort data frame based on 'concentration ratio ' column in ascending order.
#The tokens are in the top 10 for the highest ratio of appearances in robyn corpora 
#vs appearances in cher corpora.

final_count_corpora_df.sort_values('concentration_ratio', ascending =True)[0:10]

,token,corpus1 count,concentration1,corpus2 count,concentration2,concentration_ratio
235,beat,17,0.000473,146,0.009588,0.049365
180,dance,25,0.000696,150,0.009851,0.070660
290,work,15,0.000418,76,0.004991,0.083677
346,hang,8,0.000223,38,0.002496,0.089255
365,forgive,5,0.000139,16,0.001051,0.132488
356,alright,10,0.000278,30,0.001970,0.141320
90,shake,6,0.000167,18,0.001182,0.141320
361,party,8,0.000223,21,0.001379,0.161509
250,space,8,0.000223,20,0.001313,0.169585
344,pretend,6,0.000167,15,0.000985,0.169585


Q: What are some observations about the top tokens? Do you notice any interesting items on the list? 

A: Cher as a woman singer has used word of "man" in her songs more than Robyn. On Robyn lyrics we can see some words in a same category like “beat”, “dance”, “shake” and ”party”.


## Build word clouds for all four corpora. 

For building wordclouds, we'll follow exactly the code of the text. The code in this section can be found [here](https://github.com/blueprints-for-text-analytics-python/blueprints-text/blob/master/ch01/First_Insights.ipynb). If you haven't already, you should absolutely clone the repository that accompanies the book. 


In [30]:
from matplotlib import pyplot as plt

def wordcloud(word_freq, title=None, max_words=200, stopwords=None):

    wc = WordCloud(width=800, height=400, 
                   background_color= "black", colormap="Paired", 
                   max_font_size=150, max_words=max_words)
    
    # convert data frame into dict
    if type(word_freq) == pd.Series:
        counter = Counter(word_freq.fillna(0).to_dict())
    else:
        counter = word_freq

    # filter stop words in frequency counter
    if stopwords is not None:
        counter = {token:freq for (token, freq) in counter.items() 
                              if token not in stopwords}
    wc.generate_from_frequencies(counter)
 
    plt.title(title) 

    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    
    
def count_words(df, column='tokens', preprocess=None, min_freq=2):

    # process tokens and update counter
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # create counter and run through all data
    counter = Counter()
    df[column].map(update)

    # transform counter into data frame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    
    return freq_df.sort_values('freq', ascending=False)

In [37]:
robyn_count_words=count_words(lyrics_data[lyrics_data['artist']=='robyn'])
cher_count_words=count_words(lyrics_data[lyrics_data['artist']=='cher'])


In [ ]:
wordcloud(robyn_count_words['freq'], title = "Robyn Words")

In [38]:
wordcloud(cher_count_words['freq'], title = "Cher Words")

AttributeError: 'TransposedFont' object has no attribute 'getbbox'

Q: What observations do you have about these (relatively straightforward) wordclouds? 

A: 